# Module 5 - Programming Assignment

## Directions

1. Change the name of this file to be your JHED id as in `jsmith299.ipynb`. Because sure you use your JHED ID (it's made out of your name and not your student id which is just letters and numbers).
2. Make sure the notebook you submit is cleanly and fully executed. I do not grade unexecuted notebooks.
3. Submit your notebook back in Blackboard where you downloaded this file.

*Provide the output **exactly** as requested*

## Solving Normal Form Games

In [3]:
from typing import List, Tuple, Dict, Callable

In the lecture we talked about the Prisoner's Dilemma game, shown here in Normal Form:

Player 1 / Player 2  | Defect | Cooperate
------------- | ------------- | -------------
Defect  | -5, -5 | -1, -10
Cooperate  | -10, -1 | -2, -2

where the payoff to Player 1 is the left number and the payoff to Player 2 is the right number. We can represent each payoff cell as a Tuple: `(-5, -5)`, for example. We can represent each row as a List of Tuples: `[(-5, -5), (-1, -10)]` would be the first row and the entire table as a List of Lists:

In [4]:
prisoners_dilemma = [
 [( -5, -5), (-1,-10)],
 [(-10, -1), (-2, -2)]]

prisoners_dilemma

[[(-5, -5), (-1, -10)], [(-10, -1), (-2, -2)]]

in which case the strategies are represented by indices into the List of Lists. For example, `(Defect, Cooperate)` for the above game becomes `prisoners_dilemma[ 0][ 1]` and returns the payoff `(-1, -10)` because 0 is the first row of the table ("Defect" for Player 1) and 1 is the 2nd column of the row ("Cooperate" for Player 2).

For this assignment, you are going write a function that uses Successive Elimination of Dominated Strategies (SEDS) to find the **pure strategy** Nash Equilibrium of a Normal Form Game. The function is called `solve_game`:

```python
def solve_game( game: List[List[Tuple]], weak=False) -> List[Tuple]:
    pass # returns strategy indices of Nash equilibrium or None.
```

and it takes two parameters: the game, in a format that we described earlier and an optional boolean flag that controls whether the algorithm considers only **strongly dominated strategies** (the default will be false) or whether it should consider **weakly dominated strategies** as well.

It should work with game matrices of any size and it will return the **strategy indices** of the Nash Equilibrium. If there is no **pure strategy** equilibrium that can be found using SEDS, return the empty List (`[]`).


<div style="background: mistyrose; color: firebrick; border: 2px solid darkred; padding: 5px; margin: 10px;">
Do not return the payoff. That's not useful. Return the strategy indices, any other output is incorrect.
</div>

As before, you must provide your implementation in the space below, one Markdown cell for documentation and one Code cell for implementation, one function and assertations per Codecell.


---

In [ ]:
def dominated_cols(game, weak=False):
    # Extract cols
    cols = [] 
    for i in range(len(game[0])):
        col = [game[j][i][1] for j in range(len(game))]
        cols.append(col)
    
    dominated = set()
    # Compare each col to find any dominated cols
    for i in range(len(cols)):
        for j in range(len(cols)):
            if i != j:
                if weak:
                    # Weak domination: All values are less than or equal, and at least one is strictly less
                    if all(cols[i][k] <= cols[j][k] for k in range(len(cols[i]))) and any(cols[i][k] < cols[j][k] for k in range(len(cols[i]))):
                        dominated.add(i)
                else:
                    # Strict domination: All values are strictly less, and at least one is strictly greater
                    if all(cols[i][k] < cols[j][k] for k in range(len(cols[i]))):
                        dominated.add(i)
    return dominated

In [ ]:
def dominated_rows(game, weak = False):
    # Extract rows
    rows = []
    for i in range(len(game)):
        row = game[i]
        rows.append(row)

    # Compare each row to find any dominated rows    
    dominated = set()
    for i in range(len(rows)):
        for j in range(len(rows)):
            if i != j:
                if weak:
                    if all(rows[i][k][0] <= rows[j][k][0] for k in range(len(rows[i]))) and any(rows[i][k][0] < rows[j][k][0] for k in range(len(rows[i]))):
                        dominated.add(i)
                else:
                    if all(rows[i][k][0] < rows[j][k][0] for k in range(len(rows[i]))):
                        dominated.add(i)
    return dominated

In [7]:
def remove_col(game, col):
    new_game = []
    for row in game:
        new_row = [cell for i, cell in enumerate(row) if i != col]
        new_game.append(new_row)
    return new_game

In [8]:
def remove_row(game, row):
    """Remove a row from the game."""
    return [game[i] for i in range(len(game)) if i != row]

In [95]:
def generate_next_states(game, prev_move, row_map, col_map, weak=False):
    next_states = []
    if prev_move == "row":
        # Check for dominated cols
        dominated = dominated_cols(game, weak)
        if dominated:
            # print("Dominated cols:", dominated)
            for col in dominated:
                next_state = remove_col(game, col)
                next_states.append((next_state, "col", row_map, col_map[:col] + col_map[col+1:]))
    elif prev_move == "col":
        dominated = dominated_rows(game, weak)
        if dominated:
            # print("Dominated rows:", dominated)
            for row in dominated:
                next_state = remove_row(game, row)
                next_states.append((next_state, "row", row_map[:row] + row_map[row+1:], col_map))
    else:
        rows = dominated_rows(game, weak)
        cols = dominated_cols(game, weak)
        print("Rows:", rows, "Cols:", cols)
        for row in rows:
            next_state = remove_row(game, row)
            next_states.append((next_state, "row", row_map[:row] + row_map[row+1:], col_map))
        for col in cols:
            next_state = remove_col(game, col)
            next_states.append((next_state, "col", row_map, col_map[:col] + col_map[col+1:]))

    return next_states

## solve_game

(documentation)

In [132]:
import copy

def solve_game(game: List[List[Tuple]], weak: bool = False) -> List[Tuple]:
    """
    Solves the game by applying Successive Elimination of Dominated Strategies (SEDS) to find the pure strategy Nash Equilibrium of a Normal Form Game.
    :param game: A list of lists representing the payoff matrix of the game.
    :param weak: If True, applies weak dominance instead of strict dominance.
    :return: A list of tuples representing the Nash Equilibria of the game.
    """
    
    # Generate all possible states
    row_map = [i for i in range(len(game))]
    col_map = [i for i in range(len(game[0]))]
    next_states = generate_next_states(game, None, row_map, col_map, weak)
    equilibriums = set()

    while next_states:
        current_state, prev_move, row_map, col_map = next_states.pop(0)
        print(row_map, col_map)
        print("Current State:", current_state, "Previous Move:", prev_move)

        # Generate next states based on the current state
        next_states += generate_next_states(current_state, prev_move, row_map, col_map, weak)

        # Check if the current state is a Nash Equilibrium
        if len(current_state) == 1 and len(current_state[0]) == 1:
            equilibriums.add((col_map[0], row_map[0]))


    return list(equilibriums)

    

## Additional Directions

Create three games as described and according to the following:

1. Your games must be created and solved "by hand".
2. The strategy pairs must **not** be on the main diagonal (0, 0), (1, 1), or (2, 2). And the solution cannot be the same for both Game 1 and Game 2.
3. Make sure you fill out the Markdown ("?") with your game as well as the solution ("?").
4. Remember, **do not return the payoff**, return the strategy indices (a list of them).

## Before you code...

Solve the following game by hand using SEDS and weakly dominated strategies. 
The game has two (pure) Nash Equilibriums. 
You should find all of them.
This will help you think about what you need to implement to make the algorithm work.
**Hint**: You will need State Space Search from Module 1 and SEDS from Module 5 to get the full algorithm to work.

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 1/0 | 3/1 | 1/1 |
|1  | 1/1 | 3/0 | 0/1 |
|2  | 2/2 | 3/3 | 0/2 |

**Solutions**: (put them here)

In [133]:
test_1 = [
[(1, 0), (3, 1), (1, 1)],
[(1, 1), (3, 0), (0, 1)],
[(2, 2), (3, 3), (0, 2)],
]

In [134]:
solution = solve_game(test_1, weak=True)
print(solution)

Rows: {1} Cols: {0}
[0, 2] [0, 1, 2]
Current State: [[(1, 0), (3, 1), (1, 1)], [(2, 2), (3, 3), (0, 2)]] Previous Move: row
[0, 1, 2] [1, 2]
Current State: [[(3, 1), (1, 1)], [(3, 0), (0, 1)], [(3, 3), (0, 2)]] Previous Move: col
[0, 2] [1, 2]
Current State: [[(3, 1), (1, 1)], [(3, 3), (0, 2)]] Previous Move: col
[0, 2] [0, 1]
Current State: [[(1, 0), (3, 1)], [(2, 2), (3, 3)]] Previous Move: col
[0, 2] [1, 2]
Current State: [[(3, 1), (1, 1)], [(3, 3), (0, 2)]] Previous Move: row
[0, 1] [1, 2]
Current State: [[(3, 1), (1, 1)], [(3, 0), (0, 1)]] Previous Move: row
[0] [1, 2]
Current State: [[(3, 1), (1, 1)]] Previous Move: row
[2] [0, 1]
Current State: [[(2, 2), (3, 3)]] Previous Move: row
[0, 2] [1]
Current State: [[(3, 1)], [(3, 3)]] Previous Move: col
[0, 1] [2]
Current State: [[(1, 1)], [(0, 1)]] Previous Move: col
[2] [1]
Current State: [[(3, 3)]] Previous Move: col
[0] [2]
Current State: [[(1, 1)]] Previous Move: row
[(1, 2), (2, 0)]


### Test Game 1. Create a 3x3 two player game

**that can only be solved using the Successive Elimintation of Strongly Dominated Strategies**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 1/0 | 3/1 | 1/1 |
|1  | 1/1 | 3/0 | 0/1 |
|2  | 2/2 | 3/3 | 0/2 |

**Solution:** {(2, 0), (1, 2)}

In [135]:
test_game_1 = [
    [(2, 1), (1, 0), (0, 2)],
    [(3, 2), (2, 1), (1, 0)],
    [(1, 3), (0, 1), (2, 1)],
]

solution = solve_game(test_game_1, weak=False)
print(solution)

Rows: {0} Cols: {1}
[1, 2] [0, 1, 2]
Current State: [[(3, 2), (2, 1), (1, 0)], [(1, 3), (0, 1), (2, 1)]] Previous Move: row
[0, 1, 2] [0, 2]
Current State: [[(2, 1), (0, 2)], [(3, 2), (1, 0)], [(1, 3), (2, 1)]] Previous Move: col
[1, 2] [0, 2]
Current State: [[(3, 2), (1, 0)], [(1, 3), (2, 1)]] Previous Move: col
[1, 2] [0, 1]
Current State: [[(3, 2), (2, 1)], [(1, 3), (0, 1)]] Previous Move: col
[1, 2] [0, 2]
Current State: [[(3, 2), (1, 0)], [(1, 3), (2, 1)]] Previous Move: row
[1] [0, 1]
Current State: [[(3, 2), (2, 1)]] Previous Move: row
[1, 2] [0]
Current State: [[(3, 2)], [(1, 3)]] Previous Move: col
[1] [0]
Current State: [[(3, 2)]] Previous Move: col
[1] [0]
Current State: [[(3, 2)]] Previous Move: row
[(0, 1)]


In [136]:
assert solution == [(0, 1)] # insert your solution from above. TODO: change to return indices instead of values

### Test Game 2. Create a 3x3 two player game

**that can only be solved using the Successive Elimintation of Weakly Dominated Strategies**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | ? | ? | ? |
|1  | ? | ? | ? |
|2  | ? | ? | ? |

**Solution:**? (strategy indices)

In [137]:
test_game_2 = [
    [(2, 1), (3, 0), (4, 2)],
    [(1, 1), (3, 1), (3, 1)],
    [(0, 1), (1, 1), (2, 0)],
]

strong_solution = solve_game( test_game_2)
weak_solution = solve_game( test_game_2, weak=True)

Rows: {2} Cols: set()
[0, 1] [0, 1, 2]
Current State: [[(2, 1), (3, 0), (4, 2)], [(1, 1), (3, 1), (3, 1)]] Previous Move: row
Rows: {1, 2} Cols: {1}
[0, 2] [0, 1, 2]
Current State: [[(2, 1), (3, 0), (4, 2)], [(0, 1), (1, 1), (2, 0)]] Previous Move: row
[0, 1] [0, 1, 2]
Current State: [[(2, 1), (3, 0), (4, 2)], [(1, 1), (3, 1), (3, 1)]] Previous Move: row
[0, 1, 2] [0, 2]
Current State: [[(2, 1), (4, 2)], [(1, 1), (3, 1)], [(0, 1), (2, 0)]] Previous Move: col
[0, 2] [0, 2]
Current State: [[(2, 1), (4, 2)], [(0, 1), (2, 0)]] Previous Move: col
[0, 1] [1, 2]
Current State: [[(3, 0), (4, 2)], [(3, 1), (3, 1)]] Previous Move: col
[0, 1] [0, 2]
Current State: [[(2, 1), (4, 2)], [(1, 1), (3, 1)]] Previous Move: col
[0, 2] [0, 2]
Current State: [[(2, 1), (4, 2)], [(0, 1), (2, 0)]] Previous Move: row
[0, 1] [0, 2]
Current State: [[(2, 1), (4, 2)], [(1, 1), (3, 1)]] Previous Move: row
[0] [0, 2]
Current State: [[(2, 1), (4, 2)]] Previous Move: row
[0] [1, 2]
Current State: [[(3, 0), (4, 2)]] Pre

In [138]:
print("Strong Solution:", strong_solution)
print("Weak Solution:", weak_solution)

Strong Solution: []
Weak Solution: [(2, 0)]


In [140]:
assert strong_solution == []
assert weak_solution == [(2, 0)]

### Test Game 3. Create a 3x3 two player game

**that cannot be solved using the Successive Elimintation of Dominated Strategies at all**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | ? | ? | ? |
|1  | ? | ? | ? |
|2  | ? | ? | ? |

**Solution:** None

In [141]:
test_game_3 = [
    [(1, 1), (0, 2), (3, 0)],
    [(0, 0), (2, 1), (1, 3)],
    [(2, 3), (1, 0), (0, 2)],
]

strong_solution = solve_game( test_game_3)
weak_solution = solve_game( test_game_3, weak=True)

Rows: set() Cols: set()
Rows: set() Cols: set()


In [142]:
print("Strong Solution:", strong_solution)
print("Weak Solution:", weak_solution)

Strong Solution: []
Weak Solution: []


In [143]:
assert strong_solution == []
assert weak_solution == []

### Test Game 4. Multiple Equilibria

You solved the following game by hand, above.
Now use your code to solve it.

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 1/0 | 3/1 | 1/1 |
|1  | 1/1 | 3/0 | 0/1 |
|2  | 2/2 | 3/3 | 0/2 |

**Solutions:** (copy from above)

In [144]:
test_game_4 = [
[(1, 0), (3, 1), (1, 1)],
[(1, 1), (3, 0), (0, 1)],
[(2, 2), (3, 3), (0, 2)]]

strong_solution = solve_game( test_game_4)
weak_solution = solve_game( test_game_4, weak=True)

Rows: set() Cols: set()
Rows: {1} Cols: {0}
[0, 2] [0, 1, 2]
Current State: [[(1, 0), (3, 1), (1, 1)], [(2, 2), (3, 3), (0, 2)]] Previous Move: row
[0, 1, 2] [1, 2]
Current State: [[(3, 1), (1, 1)], [(3, 0), (0, 1)], [(3, 3), (0, 2)]] Previous Move: col
[0, 2] [1, 2]
Current State: [[(3, 1), (1, 1)], [(3, 3), (0, 2)]] Previous Move: col
[0, 2] [0, 1]
Current State: [[(1, 0), (3, 1)], [(2, 2), (3, 3)]] Previous Move: col
[0, 2] [1, 2]
Current State: [[(3, 1), (1, 1)], [(3, 3), (0, 2)]] Previous Move: row
[0, 1] [1, 2]
Current State: [[(3, 1), (1, 1)], [(3, 0), (0, 1)]] Previous Move: row
[0] [1, 2]
Current State: [[(3, 1), (1, 1)]] Previous Move: row
[2] [0, 1]
Current State: [[(2, 2), (3, 3)]] Previous Move: row
[0, 2] [1]
Current State: [[(3, 1)], [(3, 3)]] Previous Move: col
[0, 1] [2]
Current State: [[(1, 1)], [(0, 1)]] Previous Move: col
[2] [1]
Current State: [[(3, 3)]] Previous Move: col
[0] [2]
Current State: [[(1, 1)]] Previous Move: row


In [145]:
print("Strong Solution:", strong_solution)
print("Weak Solution:", weak_solution)

Strong Solution: []
Weak Solution: [(1, 2), (2, 0)]


In [147]:
assert strong_solution == []
assert weak_solution == [(1, 2), (2, 0)] # put solution here

## Before You Submit...

1. Did you provide output exactly as requested? **Don't forget to fill out the Markdown tables with your games**.
2. Did you re-execute the entire notebook? ("Restart Kernel and Rull All Cells...")
3. If you did not complete the assignment or had difficulty please explain what gave you the most difficulty in the Markdown cell below.
4. Did you change the name of the file to `jhed_id.ipynb`?

Do not submit any other files.